# Preprocessing

In [ ]:
!pip install -U accelerate
!pip install -q transformers datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.


In [ ]:
from datasets import load_dataset

dataset = load_dataset("csv", data_files="/content/drive/MyDrive/Code/Maitrise/sappelli_email_classification/data/numta_encoded.csv")
dataset['train'][0]

Generating train split: 0 examples [00:00, ? examples/s]

{'label': 3,
 'text': 'It was cute!\tGenia FitzGerald\t10/10/2000 03:23 PM\t\t \t\t To: Marie Heard/Enron Communications@Enron Communications\t\t cc: Tana Jones/HOU/ECT@ECT\t\t Subject: Re: HOTDOG !This is sooo adorable!Thanks for inviting me to share Mexican food with you.  Tana said it was YOUR fault I wasn t invited!!!!!Genia----- Forwarded by Genia FitzGerald/HOU/ECT on 10/10/2000 03:21 PM -----\t\t----- Forwarded by Genia FitzGerald/HOU/ECT on 10/10/2000 02:30 PM ----- - Hotdog PHOTO2.jpg'}

In [ ]:
# prompt: Split the dataset, but keep the HuggingFace format

train_test_dataset = dataset["train"].train_test_split(test_size=0.2, shuffle=False, seed=42)

# prompt: lenght of train and test

train_length = len(train_test_dataset['train'])
test_length = len(train_test_dataset['test'])

print(f"Train length: {train_length}")
print(f"Test length: {test_length}")

Train length: 456
Test length: 114


In [ ]:
train_test_dataset['test'][0]

{'label': 7,
 'text': 'I will be out tomorrow and I may leave early this afternoon.  Could you email me the deal numbers for the 3 purchases and 12 sales for the month of August?'}

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [ ]:
tokenized_dataset = train_test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/456 [00:00<?, ? examples/s]

Map:   0%|          | 0/114 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Training

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=4)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="content/drive/MyDrive/Code/Maitrise/sappelli_email_classification/results/numta",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss


TrainOutput(global_step=145, training_loss=1.2340486723801185, metrics={'train_runtime': 206.4998, 'train_samples_per_second': 11.041, 'train_steps_per_second': 0.702, 'total_flos': 599103257072640.0, 'train_loss': 1.2340486723801185, 'epoch': 5.0})

In [ ]:
trainer.train()

In [ ]:
# prompt: save model with name act1

model.save_pretrained("/content/drive/MyDrive/Code/Maitrise/sappelli_data/models/numta")


# Loading

In [ ]:
# prompt: load local model in /content/drive/MyDrive/Code/Maitrise/sappelli/models/act1/model.safetensors
from transformers import AutoModelForSequenceClassification

# Download model and configuration from huggingface.co and cache.
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=4)
# Model was saved using *save_pretrained('./test/saved_model/')* (for example purposes, not runnable).
model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/Code/Maitrise/sappelli/models/numta/")


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Evaluation

In [ ]:
print(type(train_test_dataset["test"]["label"][0]))

<class 'int'>


In [ ]:
# prompt: check if there is other type than int in test dataset

types = set()
for label in train_test_dataset["test"]["label"]:
  types.add(type(label))
if len(types) > 1:
  print("There are other types than int in test dataset")
else:
  print("All types in test dataset are int")


All types in test dataset are int


In [ ]:
results = task_evaluator.compute(model_or_pipeline=model, data=train_test_dataset["test"], metric=metric, tokenizer=tokenizer)

print(results)

ValueError: invalid literal for int() with base 10: 'LABEL_7'

In [ ]:
# prompt: evaluate the results on the test dataset, not using trainer, only model

model.eval()

# Prepare the test data
test_dataloader = data_collator.prepare_test_dataloader(tokenized_dataset["test"])

# Evaluate the model on the test data
outputs = model(**tokenized_dataset["test"])
loss, logits = outputs.loss, outputs.logits

# Compute and print the accuracy
predictions = torch.argmax(logits, dim=-1)
accuracy = (predictions == tokenized_dataset["test"]["labels"]).mean()
print(f"Accuracy on the test set: {accuracy:.3f}")


TypeError: BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplac

In [ ]:
# prompt: evaluate the results on the test dataset

evaluation = trainer.evaluate(tokenized_dataset["test"])
print(evaluation)


NameError: name 'trainer' is not defined

In [ ]:
# prompt: evaluate using f1 score

from sklearn.metrics import f1_score

# Compute and print the F1 score
f1 = f1_score(tokenized_dataset["test"]["label"], trainer.predict(tokenized_dataset["test"]).predictions.argmax(axis=-1), average="micro")
print(f"F1 score: {f1}")


KeyError: "Column labels not in the dataset. Current columns in the dataset: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask']"